In [1]:
from os.path import join
import numpy as np


def get_data(mode, label):
    base = "data/shap/undefend"
    return (
        np.load(join(base, mode, str(label), "features.npy")),
        np.load(join(base, mode, str(label), "labels.npy")),
        np.load(join(base, mode, str(label), "preds.npy")),
        np.load(join(base, mode, str(label), f"shap_values-{label}.npy")),
    )

In [2]:
features_all = {"train": [], "test": []}
labels_all = {"train": [], "test": []}
for mode in ["train", "test"]:
    for label in range(100):

        features, labels, preds, shap_values = get_data(mode, label)
        # wrong instances
        features_wrong = features[labels != preds].copy()
        # features_all.append(features_wrong)
        # right instances
        features_right = features[labels == preds].copy()
        shap_right = shap_values[labels == preds]
        assert shap_right.shape == features_right.shape

        important = features_right[shap_right > np.percentile(shap_right, 100 - 100 / 36)].copy()
        mask = important < np.random.uniform(1, 100, important.shape)
        features_right[shap_right > np.percentile(shap_right, 100 - 100 / 36)] += np.random.uniform(
            1, 10, important.shape
        ).astype(np.int32) + mask * np.random.uniform(1, 90, important.shape).astype(np.int32)
        # features_right[shap_right > 0.01] = 0
        # features_all.append(features_right)
        # labels_all.append(labels.copy())
        features_all[mode].append(features_wrong)
        features_all[mode].append(features_right)
        labels_all[mode].append(labels.copy())
    features_all[mode] = np.concatenate(features_all[mode], axis=0)
    labels_all[mode] = np.concatenate(labels_all[mode], axis=0)
# features_all = np.concatenate(features_all, axis=0)
# labels_all = np.concatenate(labels_all, axis=0)
np.savez(
    "data/shap/deepexplainer-new.npz",
    features_train=features_all["train"],
    labels_train=labels_all["train"],
    features_test=features_all["test"],
    labels_test=labels_all["test"],
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()
ax.hist(tmp[tmp > 0].flatten(), bins=10 ** (np.arange(0, 10)) / 10**10)
# log scale
ax.set_xscale("log")
plt.show()